In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install imblearn
!pip install collectoins

ERROR: Could not find a version that satisfies the requirement collectoins (from versions: none)
ERROR: No matching distribution found for collectoins


In [3]:
data=pd.read_csv("Final Transactions.csv")
data.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,0,2023-01-01 00:00:31,596,3156,533.07,31,0,0,0
1,1,1,2023-01-01 00:02:10,4961,3412,808.56,130,0,0,0
2,2,2,2023-01-01 00:07:56,2,1365,1442.94,476,0,1,1
3,3,3,2023-01-01 00:09:29,4128,8737,620.65,569,0,0,0
4,4,4,2023-01-01 00:10:34,927,9906,490.66,634,0,0,0


In [4]:
data.drop(['Unnamed: 0','TRANSACTION_ID','TX_TIME_SECONDS','TX_TIME_DAYS'],axis=1,inplace=True)
data.head()

,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TX_FRAUD_SCENARIO
0,2023-01-01 00:00:31,596,3156,533.07,0,0
1,2023-01-01 00:02:10,4961,3412,808.56,0,0
2,2023-01-01 00:07:56,2,1365,1442.94,1,1
3,2023-01-01 00:09:29,4128,8737,620.65,0,0
4,2023-01-01 00:10:34,927,9906,490.66,0,0


In [13]:
data['TX_DATETIME']=pd.to_datetime(data['TX_DATETIME'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754155 entries, 0 to 1754154
Data columns (total 6 columns):
 #   Column             Dtype         
---  ------             -----         
 0   TX_DATETIME        datetime64[ns]
 1   CUSTOMER_ID        int64         
 2   TERMINAL_ID        int64         
 3   TX_AMOUNT          float64       
 4   TX_FRAUD           int64         
 5   TX_FRAUD_SCENARIO  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 80.3 MB


In [14]:
#convert dateime to year,month and day
data['year'] = data['TX_DATETIME'].dt.year
data['month'] = data['TX_DATETIME'].dt.month
data['day'] = data['TX_DATETIME'].dt.day

In [17]:
data.drop('TX_DATETIME',axis=1,inplace=True)
data.head()

,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TX_FRAUD_SCENARIO,year,month,day
0,596,3156,533.07,0,0,2023,1,1
1,4961,3412,808.56,0,0,2023,1,1
2,2,1365,1442.94,1,1,2023,1,1
3,4128,8737,620.65,0,0,2023,1,1
4,927,9906,490.66,0,0,2023,1,1


In [18]:
x=data.iloc[:,[0,1,2,5,6,7]].values
y=data.iloc[:,3].values

from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2)

In [19]:
# Over sampling
from imblearn.over_sampling import SMOTE
from collections import Counter

print('Data before Oversampling',Counter(train_y))

Data before Oversampling Counter({0: 1214679, 1: 188645})


In [20]:
sm=SMOTE()
x_res,y_res=sm.fit_resample(train_x,train_y)
print("Data after Oversampling",Counter(y_res))

Data after Oversampling Counter({0: 1214679, 1: 1214679})


In [25]:
# Model (Decision tree)
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(max_depth=3)
model.fit(x_res,y_res)

DecisionTreeClassifier(max_depth=3)

In [22]:
# Evaluation

pred_y=model.predict(test_x)
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

print('Accuracy:',accuracy_score(pred_y,test_y))
print('F1 score:',f1_score(pred_y,test_y))
print('Recall score:',recall_score(pred_y,test_y))
print('Precision score:',precision_score(pred_y,test_y))


Accuracy: 0.9952398733293238
F1 score: 0.9820387618576438
Recall score: 1.0
Precision score: 0.9647113515341054


In [23]:
# Evaluate

from sklearn.metrics import confusion_matrix
print('confusion_matrix:\n',confusion_matrix(pred_y,test_y))

confusion_matrix:
 [[303507   1670]
 [     0  45654]]


In [26]:
# Check overfitting

pred_y=model.predict(x_res)
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

print('Accuracy:',accuracy_score(pred_y,y_res))
print('F1 score:',f1_score(pred_y,y_res))
print('Recall score:',recall_score(pred_y,y_res))
print('Precision score:',precision_score(pred_y,y_res))


Accuracy: 0.9840513419594806
F1 score: 0.9837928598229826
Recall score: 1.0
Precision score: 0.9681026839189613
